<a href="https://colab.research.google.com/github/Linajbjdfh/Data-Engineering-and-Machine-Learning-Operations/blob/main/Assignment_1_Steam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

# Load the data into a pandas dataframe
df = pd.read_csv("/content/steam_reviews.csv", encoding= 'unicode_escape')

In [ ]:
df.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title
0,2019-02-10,2,4,578,False,Recommended,&gt Played as German Reich&gt Declare war on B...,Expansion - Hearts of Iron IV: Man the Guns
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight


In [ ]:
# Importing the necessary libraries
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect('steam_reviews.db')

# Add a column for the sentiment labels
df['sentiment'] = ''

In [ ]:
df.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,sentiment
0,2019-02-10,2,4,578,False,Recommended,&gt Played as German Reich&gt Declare war on B...,Expansion - Hearts of Iron IV: Man the Guns,
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,


In [ ]:
# Load the DataFrame into the SQLite table
df.to_sql('reviews', conn, if_exists='append', index=False)

11528

In [ ]:
df.shape

(11528, 9)

In [ ]:
# Select all records from the 'reviews' table
select_query = "SELECT review FROM reviews limit 100;"
cursor = conn.execute(select_query)
rows = cursor.fetchall()

# Print the records
for row in rows:
    print(row)

("&gt Played as German Reich&gt Declare war on Belgium&gt Can't break Belgium so go through France&gt Capitulate France in order to get to Belgium&gt Get True Blitzkrieg achievementThis game is dad",)
('yes.',)
("Very good game although a bit overpriced in my opinion. I'd prefer playing the game with mods (historical accuracy so on) although the vanilla version is good aswell. 7/10",)
("Out of all the reviews I wrote This one is probably the most serious one I wrote. For starters the community of this game sucks just like every online game You don't wanna talk to them because they're all just jerks who wanna say you suck or you camped or blah blah blah its rare to get a nice compliment out of people in this game.But you don't have to talk to the community either. Unlike most online games this one doesn't have a mic system so you don't have to worry about hearing a twelve year old cry baby raging at you for nothing. Only time you can talk to people in the game is in the beginning before

In [ ]:
!pip install transformers --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.3 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

# Load the pre-trained sentiment analysis model
classifier = pipeline('sentiment-analysis')


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# Extract sentiment reviews for the reviews
reviews = conn.execute('SELECT * FROM reviews limit 100')

In [ ]:
# Iterate over the recipeies and update the sentiment label for each one
for i, row in enumerate(reviews):
    # Extract the text of the current review
    reviews = row[0]
    
    # Analyze the sentiment of the review using the pre-trained classifier
    sentiment = classifier(reviews[:12000])[0]['label']
    
    # Map the sentiment label to a binary label (1 for positive, 0 for negative)
    if sentiment == 'POSITIVE':
      label = 1
    else:
      label = 0
      
    # Update the 'sentiment' column in the 'reviews' table with the binary label for the current review
    conn.execute('UPDATE reviews SET sentiment = ? WHERE rowid = ?', (label, i+1))
    
# Commit the changes to the database
conn.commit()


In [ ]:
# Define the SQL query
query = 'SELECT * FROM reviews LIMIT 100'

# Execute the query and convert the result to a DataFrame
df_q = pd.read_sql_query(query, conn)
df_q

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,sentiment
0,2019-02-10,2,4,578,0,Recommended,&gt Played as German Reich&gt Declare war on B...,Expansion - Hearts of Iron IV: Man the Guns,1
1,2019-02-10,0,0,184,0,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,1
2,2019-02-07,0,0,892,0,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,1
3,2018-06-14,126,1086,676,0,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,1
4,2017-06-20,85,2139,612,0,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,1
...,...,...,...,...,...,...,...,...,...
95,2017-11-01,0,0,22,0,Recommended,I have not played this game for very long but ...,Dead by Daylight,1
96,2016-06-14,1,1,322,0,Recommended,Dead by Daylight is a very interesting game. B...,Dead by Daylight,1
97,2018-11-24,0,0,822,0,Recommended,so after 700+ hrs of playing this game i still...,Dead by Daylight,1
98,2016-06-14,0,0,8,0,Recommended,Very fun game so far. Great atmosphere that re...,Dead by Daylight,1


In [ ]:
from transformers import pipeline
from sklearn.metrics import accuracy_score

# Load the data from the SQLite database
X = pd.read_sql_query('SELECT review FROM reviews limit 100', conn)
y = pd.read_sql_query('SELECT sentiment FROM reviews limit 100', conn)

# Train a logistic regression model on the sentiment labels
clf = pipeline('sentiment-analysis')
y_pred = [int(result['label'] == 'POSITIVE') for result in clf(X['review'].to_list(), truncation=True)]

# Evaluate the model on the testing set
accuracy = accuracy_score(y['sentiment'].astype(int).to_list(), y_pred)
print(f'Accuracy: {accuracy:.2f}')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Accuracy: 0.58


In [ ]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 94.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.5 MB/s eta 0

In [ ]:
import gradio as gr

In [ ]:
demo = gr.Interface(
    fn=clf, 
    inputs=gr.Textbox(placeholder="Copy and paste/Write a game review here..."), 
    outputs="label", 
    interpretation="default",
    examples=[["This game is good"], ["This game is bad"]],
    title="Sentiment Analysis App",
    description="This Application assesses if a game review is positive or negative.", )

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>